In [46]:
import datetime
import json
from bs4 import BeautifulSoup
import requests
from dataclasses import dataclass, field
import types

@dataclass
class Holiday:
    name: str
    date: datetime
    
    def __str__(self):
        return f"{self.name} ({self.date})"

In [86]:
class HolidayList:
    "HolidayList class"
    def __init__(self):
        self.innerHolidays = []

    def addHoliday(self, holidayObj):
        if isinstance(holidayObj, Holiday):
            self.innerHolidays.append(holidayObj)
            print(f"Added holiday {holidayObj}")

    def findHoliday(self, HolidayName, Date):
        for holiday in self.innerHolidays:
            if holiday.name == HolidayName and holiday.date == Date:
                return holiday
            else:
                print(f"{holiday} is not in list.")

    def removeHoliday(self, HolidayName, Date):
        for holiday in self.innerHolidays:
            if holiday.name == HolidayName and holiday.date == Date:
                self.innerHolidays.remove(holiday)
                print(f"{holiday} was removed from the list.")
            else:
                print(f"{holiday} is not in list.")

In [88]:
holiday1 = Holiday("New Year's Day", "2022-01-01")
#print(holiday1)

holidaylist = HolidayList()

holidaylist.addHoliday(holiday1)

holidaylist.findHoliday("New Year's Day", "2022-01-01")

holidaylist.removeHoliday("New Year's Day", "2022-01-01")



New Year's Day (2022-01-01) was removed from the list.


In [4]:
import requests

url = "https://community-open-weather-map.p.rapidapi.com/forecast/daily"

querystring = {"q":"new york,us","lat":"41","lon":"74","cnt":"7","units":"imperial"}

headers = {
    'x-rapidapi-host': "community-open-weather-map.p.rapidapi.com",
    'x-rapidapi-key': "1e19830f3bmshf56abb61a6e9168p189818jsnf035a12afc26"
    }

weather = requests.request("GET", url, headers=headers, params=querystring)
weather_dict = weather.json()

print(weather_dict)

print(weather_dict["list"][0]["weather"][0]["main"])

{'city': {'id': 5128581, 'name': 'New York', 'coord': {'lon': -74.006, 'lat': 40.7143}, 'country': 'US', 'population': 8175133, 'timezone': -18000}, 'cod': '200', 'message': 18.5456298, 'cnt': 7, 'list': [{'dt': 1641661200, 'sunrise': 1641644389, 'sunset': 1641678315, 'temp': {'day': 26.92, 'min': 21.06, 'max': 28.71, 'night': 27.9, 'eve': 25.93, 'morn': 21.06}, 'feels_like': {'day': 19.38, 'night': 18.95, 'eve': 20.23, 'morn': 8.46}, 'pressure': 1035, 'humidity': 53, 'weather': [{'id': 800, 'main': 'Clear', 'description': 'sky is clear', 'icon': '01d'}], 'speed': 15.7, 'deg': 303, 'gust': 30.58, 'clouds': 0, 'pop': 0}, {'dt': 1641747600, 'sunrise': 1641730779, 'sunset': 1641764775, 'temp': {'day': 36.46, 'min': 27.68, 'max': 37.96, 'night': 37.96, 'eve': 36.14, 'morn': 31.12}, 'feels_like': {'day': 26.56, 'night': 30.36, 'eve': 28.17, 'morn': 22.23}, 'pressure': 1023, 'humidity': 73, 'weather': [{'id': 500, 'main': 'Rain', 'description': 'light rain', 'icon': '10d'}], 'speed': 17.4, '